# Classification Algorithms: Theory, Math, and Implementation

Classification is a supervised learning technique used to predict discrete class labels based on input features. In this notebook, we will explore five fundamental classification algorithms, understand their mathematical foundations, and visualize their decision boundaries using Python and `scikit-learn`.

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from ipywidgets import interact, IntSlider, FloatSlider, FloatLogSlider

plt.style.use('seaborn-v0_8-darkgrid')

np.random.seed(42)
X_class, y_class = make_classification(n_samples=300, n_features=2, n_informative=2, 
                                       n_redundant=0, n_clusters_per_class=1, 
                                       flip_y=0.1, class_sep=1.2, random_state=42)

def plot_decision_boundary(model, X, y, title):
    """Helper function to plot interactive decision boundaries."""
    h = .02  # step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(8, 5))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', cmap='coolwarm', s=40)
    plt.title(title, fontsize=14)
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.show()

### 1. Logistic Regression
Despite its name, Logistic Regression is a **classification** algorithm. It uses a logistic (sigmoid) function to map predictions to probabilities (between 0 and 1) for binary classification.

**Mathematical Foundation:**
It transforms the output of a linear equation using the Sigmoid function $\sigma(z)$:
$$P(y=1|X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \dots + \beta_n x_n)}}$$

Where:
* $P(y=1|X)$ is the probability that the data point belongs to class 1.
* $\beta_0, \beta_1, \dots$ are the learned weights.

**Example Problem:**
* **Medicine:** Predicting if a tumor is Malignant (1) or Benign (0) based on continuous features like size and texture.

The slider below controls `C`, the inverse of regularization strength. Smaller values specify stronger regularization, forcing a simpler decision boundary.

In [2]:
@interact(C=FloatLogSlider(value=1.0, min=-3, max=3, step=0.5, description='C (1/Penalty)'))
def plot_logistic_regression(C):
    log_reg = LogisticRegression(C=C, random_state=42)
    log_reg.fit(X_class, y_class)
    plot_decision_boundary(log_reg, X_class, y_class, f"Logistic Regression (C={C:.2e})")

interactive(children=(FloatLogSlider(value=1.0, description='C (1/Penalty)', max=3.0, min=-3.0, step=0.5), Out…

### 2. K-Nearest Neighbors (K-NN)
K-NN is an intuitive, instance-based learning algorithm. It classifies a new data point based on the majority class of its 'k' nearest neighbors in the feature space.

**Mathematical Foundation:**
To find the "nearest" neighbors, it relies on distance metrics. The most common is the Euclidean Distance between two points $p$ and $q$:
$$d(p, q) = \sqrt{\sum_{i=1}^{n}(q_i - p_i)^2}$$

**Example Problem:**
* **Pattern Recognition:** Recognizing handwritten digits (0-9) based on the pixel similarity to known, labeled digit images.

Use the slider to change `k` (the number of neighbors). Notice how a very low `k` creates highly fragmented boundaries (overfitting to noise), while a high `k` creates very smooth, generalized boundaries.

In [3]:
@interact(n_neighbors=IntSlider(min=1, max=50, step=1, value=5, description='k neighbors'))
def plot_knn(n_neighbors):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_class, y_class)
    plot_decision_boundary(knn, X_class, y_class, f"K-Nearest Neighbors (k={n_neighbors})")

interactive(children=(IntSlider(value=5, description='k neighbors', max=50, min=1), Output()), _dom_classes=('…

### 3. Support Vector Machines (SVM)

SVM is a powerful classifier that finds the optimal hyperplane that best separates different classes while maximizing the margin (distance) between the classes' closest points (the support vectors).

**Mathematical Foundation:**
For linearly separable data, the goal is to maximize the margin $\frac{2}{||w||}$. This is framed as a minimization problem:
$$\text{Minimize: } \frac{1}{2} ||w||^2$$
$$\text{Subject to: } y_i(\langle w, x_i \rangle + b) \ge 1 \text{ for all } i$$

For non-linear data, SVM uses the **Kernel Trick** (like Radial Basis Function - RBF) to map data into higher dimensions where a hyperplane can separate it.

**Example Problem:**
* **Neuroscience:** Classifying EEG signals to detect if a subject is in a "Resting" vs. "Active" state.

**Hyperparameters:**
* **C (Penalty):** High C creates a strict boundary with fewer margin violations. Low C encourages a wider, softer margin.
* **Gamma:** Defines how far the influence of a single training example reaches. High gamma creates tight, complex boundaries around individual points.

In [4]:
@interact(C=FloatLogSlider(value=1.0, min=-2, max=3, step=0.5, description='C (Penalty)'),
          gamma=FloatLogSlider(value=0.1, min=-3, max=1, step=0.5, description='Gamma'))
def plot_svm(C, gamma):
    svm_clf = SVC(kernel='rbf', C=C, gamma=gamma, random_state=42)
    svm_clf.fit(X_class, y_class)
    plot_decision_boundary(svm_clf, X_class, y_class, f"SVM RBF (C={C:.2f}, Gamma={gamma:.3f})")

interactive(children=(FloatLogSlider(value=1.0, description='C (Penalty)', max=3.0, min=-2.0, step=0.5), Float…

### 4. Naive Bayes
Naive Bayes is a fast, probabilistic classifier based on Bayes' Theorem. It relies on the "naive" assumption of conditional independence between every pair of features given the class label.

**Mathematical Foundation:**
Based on Bayes' theorem, the probability of class $y$ given a set of features $X = (x_1, \dots, x_n)$ is:
$$P(y \mid x_1, \dots, x_n) = \frac{P(x_1, \dots, x_n \mid y) P(y)}{P(x_1, \dots, x_n)}$$

Assuming independent features, this simplifies to maximizing the numerator:
$$\hat{y} = \arg\max_{y} P(y) \prod_{i=1}^{n} P(x_i \mid y)$$

**Example Problem:**
* **Natural Language Processing (NLP):** Classifying emails as "Spam" or "Inbox" based on the frequency of certain words (e.g., "Free", "Winner", "Meeting").

The slider below controls `var_smoothing`, which artificially adds a tiny portion of the largest variance to all features to prevent division-by-zero errors and mathematically stabilize the curve.

In [5]:
@interact(var_smoothing=FloatLogSlider(value=1e-9, min=-10, max=-1, step=1, description='Smoothing'))
def plot_naive_bayes(var_smoothing):
    nb_clf = GaussianNB(var_smoothing=var_smoothing)
    nb_clf.fit(X_class, y_class)
    plot_decision_boundary(nb_clf, X_class, y_class, f"Gaussian Naive Bayes (Smoothing={var_smoothing:.1e})")

interactive(children=(FloatLogSlider(value=1e-09, description='Smoothing', max=-1.0, min=-10.0, step=1.0), Out…

### 5. Decision Tree Classification

A Decision Tree classifies data by recursively splitting the dataset based on feature values. It creates a flowchart-like tree structure by asking a sequence of true/false questions.

**Mathematical Foundation:**
The tree splits nodes by finding the feature and threshold that minimize **Impurity**. A common metric is the **Gini Impurity**:
$$Gini(Q) = 1 - \sum_{k=1}^{C} p_{k}^2$$
Where $p_k$ is the ratio of class $k$ instances among the training instances in node $Q$. The tree tries to make each resulting leaf node as homogeneous (pure) as possible.

**Example Problem:**
* **Medical Diagnosis:** Diagnosing a disease based on a checklist of symptoms (e.g., Does patient have fever? Yes $\rightarrow$ Cough? Yes $\rightarrow$ Flu).

The slider below controls `max_depth`. A shallow tree may underfit the data, but an unrestricted deep tree will memorize the training data (overfitting) by creating microscopic decision boundaries.

In [6]:
@interact(max_depth=IntSlider(min=1, max=15, step=1, value=3, description='Max Depth'))
def plot_decision_tree(max_depth):
    tree_clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    tree_clf.fit(X_class, y_class)
    plot_decision_boundary(tree_clf, X_class, y_class, f"Decision Tree (Max Depth={max_depth})")

interactive(children=(IntSlider(value=3, description='Max Depth', max=15, min=1), Output()), _dom_classes=('wi…